In [1]:
import numpy as np
from holisticai.datasets import load_dataset
from holisticai.security.attackers.attribute_inference.wrappers.classification.scikitlearn import ScikitlearnDecisionTreeClassifier
from holisticai.security.attackers.attribute_inference.mitigation.attacks.white_box_lifestyle_decision_tree import (
    AttributeInferenceWhiteBoxLifestyleDecisionTree
)

In [2]:
dataset = load_dataset('german_credit', preprocessed=True)
train_test = dataset.train_test_split(test_size=0.2, stratify=dataset['y'], random_state=0)
train = train_test['train']
test = train_test['test']
train

{"dtype":"Dataset","attributes":{"Instances":800,"Features":["X , y , p_attrs"]},"metadata":null}

In [3]:
job_feat_train = train['X']['Job'].values
job_feat_train[job_feat_train < 2] = 0
job_feat_train[job_feat_train >= 2] = 1

job_feat_test = test['X']['Job'].values
job_feat_test[job_feat_test < 2] = 0
job_feat_test[job_feat_test >= 2] = 1

In [4]:
unique_values = np.unique(job_feat_test, return_counts=True)
unique_values, len(job_feat_test)

((array([0, 1]), array([ 46, 154])), 200)

In [5]:
x_train = train['X'].drop(columns=['Credit amount', 'Duration']).values
x_test = test['X'].drop(columns=['Credit amount', 'Duration']).values

y_train = train['y'].values
y_test = test['y'].values

In [6]:
attack_feature = 0
x_train[:, attack_feature] = job_feat_train
x_test[:, attack_feature] = job_feat_test

In [7]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
classifier = ScikitlearnDecisionTreeClassifier(classifier)

attack = AttributeInferenceWhiteBoxLifestyleDecisionTree(estimator=classifier, attack_feature=attack_feature)

attack_x_test = np.delete(x_test, attack_feature, axis=1)

feat_true = x_test[:, attack_feature]

values = [0, 1]
priors = [46/200, 154/200]

feat_pred = attack.infer(attack_x_test, values=values, priors=priors)

In [8]:
from holisticai.efficacy.metrics import classification_efficacy_metrics

classification_efficacy_metrics(feat_true, feat_pred)

,Value,Reference
Metric,,
Accuracy,0.735000,1
Balanced Accuracy,0.500141,1
Precision,0.770053,1
Recall,0.935065,1
F1-Score,0.844575,1
